In [ ]:
#| default_exp bridge_plugins

In [ ]:
#| export
from __future__ import annotations

# Bridge plugins
> More examples of bridge plugins.

* Python wrapper of HTMX API.
* NBHooks wrapping notebook hooks.
* ZeroMD for markdown rendering.

Here I use pure python/IPython facilities to help us control, inspect and modify cell outputs.   
In subsequent notebooks I'll add more facilities to help us capture the current state of the notebook (21_nbstate.ipynb) and make the link bidirectional (32_bridget.ipynb).

In [ ]:
import ipywidgets as W
W.Widget.close_all()  # we don't want stale widgets around when developing

# Prologue

In [ ]:
#| export
from typing import Literal
from typing import TypeAlias

import fastcore.all as FC
import traitlets as T
from fastcore.xml import to_xml
from fasthtml.core import FT
from fasthtml.xtend import Script
from IPython.display import DisplayHandle
from olio.basic import bundle_path
from olio.common import update_
from traitlets import Any


In [ ]:
#| export
import bridget.fasthtml_patching
import bridget.nb_hooks
from bridget.bridge import BridgePlugin
from bridget.bridge import BridgeWidget
from bridget.bridge import debug
from bridget.bridge_widget import anysource
from bridget.bridge_widget import bundled
from bridget.helpers import id_gen
from bridget.nb_hooks import get_bridged
from bridget.nb_hooks import get_capturer
from bridget.nb_hooks import get_csi
from bridget.nb_hooks import get_nb_from_hooks


In [ ]:
import inspect
import os
import random

import fasthtml.components as ft
from fastcore.test import *
from fasthtml.components import highlight
from fasthtml.components import showtags
from fasthtml.core import fh_cfg
from IPython.display import display
from IPython.core.display import Image
from IPython.core.display import JSON
from IPython.core.display import Markdown
from olio.common import AD
from olio.common import setup_console
from olio.common import shorten
from olio.common import shortens
from olio.widget import cleanupwidgets


In [ ]:
from bridget.bridge import BLogger
from bridget.bridge import get_bridge
from bridget.bridge import ScriptsDetails
from bridget.bridge_widget import _show
from bridget.bridge_widget import ablocks
from bridget.bridge_widget import blocks
from bridget.helpers import bridge_cfg
from bridget.helpers import DEBUG
from bridget.helpers import DetailsJSON
from bridget.helpers import displaydh
from bridget.helpers import HTML
from bridget.nb import NB


In [ ]:
from fasthtml.components import Div, P, Pre, Text, Span, show, B, Details, Summary, Pre, A, Br

----


In [ ]:
# hate stupid wiggly reds
__cellinfo__:AD
__lastcellinfo__:AD
__nb__: NB

In [ ]:
os.environ['DEBUG_BRIDGET'] = 'True'

In [ ]:
#| exporti

BUNDLE_PATH = bundle_path(__name__)
new_id = id_gen()
_n = '\n'

In [ ]:
console, cprint = setup_console(140)
print(f"{DEBUG()=}")
bridge_cfg.auto_show=True
show(DetailsJSON(bridge_cfg.as_dict()))

DEBUG()=True


In [ ]:
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

In [ ]:
bridge = get_bridge(show_logger=True, wait=5)

moduleName='./bcanvas.js' buffers=[]
moduleName='./fcanvas.js' buffers=[]
moduleName='./bridge.js' buffers=[]
moduleName='./observer.js' buffers=[]
moduleName='./commander.js' buffers=[]


In [ ]:
#| exporti

commander_js = BUNDLE_PATH / 'js/commander.js'
commander_esm = bundled(commander_js)()

# HTMX API

### Method - `htmx.swap()` {#swap}

Performs swapping (and settling) of HTML content

##### Parameters

* `target` - the HTML element or string selector of swap target
* `content` - string representation of content to be swapped
* `swapSpec` - swapping specification, representing parameters from `hx-swap`
  * `swapStyle` (required) - swapping style (`innerHTML`, `outerHTML`, `beforebegin` etc)
  * `swapDelay`, `settleDelay` (number) - delays before swapping and settling respectively
  * `transition` (bool) - whether to use HTML transitions for swap
  * `ignoreTitle` (bool) - disables page title updates
  * `head` (string) - specifies `head` tag handling strategy (`merge` or `append`). Leave empty to disable head handling
  * `scroll`, `scrollTarget`, `show`, `showTarget`, `focusScroll` - specifies scroll handling after swap
* `swapOptions` - additional *optional* parameters for swapping
  * `select` - selector for the content to be swapped (equivalent of `hx-select`)
  * `selectOOB` - selector for the content to be swapped out-of-band (equivalent of `hx-select-oob`)
  * `eventInfo` - an object to be attached to `htmx:afterSwap` and `htmx:afterSettle` elements
  * `anchor` - an anchor element that triggered scroll, will be scrolled into view on settle. Provides simple alternative to full scroll handling
  * `contextElement` - DOM element that serves as context to swapping operation. Currently used to find extensions enabled for specific element
  * `afterSwapCallback`, `afterSettleCallback` - callback functions called after swap and settle respectively. Take no arguments


##### Example

```js
    // swap #output element inner HTML with div element with "Swapped!" text
    htmx.swap("#output", "<div>Swapped!</div>", {swapStyle: 'innerHTML'});
```


In [ ]:
#| export

SwapStyleT: TypeAlias = Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin',
                        'beforeend','afterend','delete','none']

In [ ]:
#| exporti

def swap(self, 
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: SwapStyleT='innerHTML',
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ):
    if isinstance(content, FT): content = to_xml(content)
    elif hasattr(content, 'to_html'): content = content.to_html()
    d = {
        'target': f"#{target}" if isinstance(target, FT) else target,
        'content': content,
        'swapSpec': update_(**{
            'swapStyle': swapStyle, 'swapDelay': swapDelay, 'settleDelay': settleDelay,
            'transition': transition,
            # 'ignoreTitle': ignoreTitle, 'head': head,
            'scroll': scroll, 'scrollTarget': scrollTarget,
            'show': show, 'showTarget': showTarget, 'focusScroll': focusScroll,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
        'swapOptions': update_(**{
            'select': select, 'selectOOB': selectOOB,
            # 'eventInfo': eventInfo,
            'anchor': anchor,
            # 'contextElement': contextElement,
            # 'afterSwapCallback': afterSwapCallback, 'afterSettleCallback': afterSettleCallback,
        }),
    }
    self.send({
        'ctx': 'commander',
        'cmd': 'swap',
        'args': [*d.values()]
    })


# HTMX Commander
> Python wrapper of HTMX API.

Currently only implemented `htmx.swap()` manually. If useful, I'll automate wrappers generation from htmx docs.


In [ ]:
commander_esm = bundled(commander_js)(debugger=DEBUG(), ts=True)

In [ ]:
#| export

class HTMXCommander(BridgeWidget):
    _esm = anysource(commander_esm, '''
export default { 
    async initialize({ model }) {
        await setupCommander(model.get('output_sels'));
        model.on("msg:custom", onCommanderMsg);
        model.set('_loaded', true); model.save_changes();
        return () => { 
            model.off("msg:custom", onCommanderMsg);
            model.set('_loaded', false); model.save_changes(); 
        };
    }
};
''')
    
    output_sels = T.List(['.output', '.jp-Cell-outputArea']).tag(sync=True)

    ctx_name = T.Unicode('commander').tag(sync=True)

    # hate stupid wiggly reds
    def swap(self: HTMXCommander,
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin','beforeend','afterend','delete','none'],
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ): ...

In [ ]:
cleanupwidgets('cmdr')

cmdr = HTMXCommander.create(timeout=2)#DEBUG(2))

In [ ]:
#| export

FC.patch_to(HTMXCommander)(swap)

## Test swap

In [ ]:
%%HTML

<div id="output-99">Original</div>

In [ ]:
cmdr.swap('#output-99', '<div>Swapped!</div>', swapStyle='innerHTML')

In [ ]:
cmdr.close()

## ----
First steps exploring automation. Skip it.

In [ ]:
sig = inspect.signature(cmdr.swap)
print(sig)
print(sig.parameters)
target_param = sig.parameters['swapStyle']
target_param.name, target_param.default, target_param.annotation, target_param.kind

(target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)
OrderedDict({'target': <Parameter "target">, 'content': <Parameter "content">, 'swapStyle': <Parameter "swapStyle: 'SwapStyleT' = 'innerHTML'">, 'swapDelay': <Parameter "swapDelay: 'int | None' = None">, 'settleDelay': <Parameter "settleDelay: 'int | None' = None">, 'transition': <Parameter "transition: 'bool | None' = None">, 'scroll': <Parameter "scroll: 'str | None' = None">, 'scrollTarget': <Parameter "scrollTarget: 'str | None' = None">, 'show': <Parameter "show: 'str | None' = None">, 'showTarget': <Parameter "showTarget: 'str | None' = None">, 'focusScroll': <Parameter "focusSc

('swapStyle', 'innerHTML', 'SwapStyleT', <_ParameterKind.KEYWORD_ONLY: 3>)

In [ ]:
bb = sig.bind(target='#target', content='<content />', swapStyle='innerHTML', swapDelay=1000)
bb.arguments, bb.signature, bb.args, bb.kwargs

({'target': '#target',
  'content': '<content />',
  'swapStyle': 'innerHTML',
  'swapDelay': 1000},
 <Signature (target, content, *, swapStyle: 'SwapStyleT' = 'innerHTML', swapDelay: 'int | None' = None, settleDelay: 'int | None' = None, transition: 'bool | None' = None, scroll: 'str | None' = None, scrollTarget: 'str | None' = None, show: 'str | None' = None, showTarget: 'str | None' = None, focusScroll: 'bool | None' = None, select: 'str | None' = None, selectOOB: 'str | None' = None, anchor: 'str | None' = None)>,
 ('#target', '<content />'),
 {'swapStyle': 'innerHTML', 'swapDelay': 1000})

In [ ]:
swap_args = {
    'target': 'target',
    'content': 'content',
    '*': '*',
    'swapSpec': {
        'swapStyle': 'swapStyle', 'swapDelay': 'swapDelay', 'settleDelay': 'settleDelay',
        'transition': 'transition',
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': 'scroll', 'scrollTarget': 'scrollTarget',
        'show': 'show', 'showTarget': 'showTarget', 'focusScroll': 'focusScroll',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}

In [ ]:
from inspect import Parameter


swap_args = {
    'target': Parameter('target', Parameter.POSITIONAL_OR_KEYWORD),
    'content': Parameter('content', Parameter.POSITIONAL_OR_KEYWORD),
    'swapSpec': {
        'swapStyle': Parameter('swapStyle', Parameter.KEYWORD_ONLY, annotation=SwapStyleT), 
        'swapDelay': Parameter('swapDelay', Parameter.KEYWORD_ONLY), 
        'settleDelay': Parameter('settleDelay', Parameter.KEYWORD_ONLY),
        'transition': Parameter('transition', Parameter.KEYWORD_ONLY),
        # 'ignoreTitle': 'ignoreTitle', 'head': 'head',
        'scroll': Parameter('scroll', Parameter.KEYWORD_ONLY), 
        'scrollTarget': Parameter('scrollTarget', Parameter.KEYWORD_ONLY),
        'show': Parameter('show', Parameter.KEYWORD_ONLY), 
        'showTarget': Parameter('showTarget', Parameter.KEYWORD_ONLY), 
        'focusScroll': Parameter('focusScroll', Parameter.KEYWORD_ONLY),
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
    'swapOptions': {
        'select': 'select', 'selectOOB': 'selectOOB',
        # 'eventInfo': 'eventInfo',
        'anchor': 'anchor',
        # 'contextElement': 'contextElement',
        # 'afterSwapCallback': 'afterSwapCallback', 'afterSettleCallback': 'afterSettleCallback',
    },
}

In [ ]:
def build_args_dict(args_desc: dict, *args, **kwargs) -> dict:
    "Builds a dictionary from args/kwargs based on an arguments descriptor"
    result = {}
    
    # Process descriptor items in order
    for k, v in args_desc.items():
        if isinstance(v, dict):
            # Handle nested dictionary descriptors
            nested = {
                v2: kwargs.get(k2) 
                for k2, v2 in v.items() 
                if kwargs.get(k2) is not None
            }
            if nested: result[k] = nested
        else:
            # For non-dict values, first try kwargs, then fallback to positional args
            if k in kwargs:
                result[v] = kwargs[k]
            elif len(args) > 0:
                result[v] = args[0]
                args = args[1:]
    
    return result


build_args_dict(swap_args, '#target', '<content />', swapStyle='innerHTML', swapDelay=1000)

{<Parameter "target">: '#target',
 <Parameter "content">: '<content />',
 'swapSpec': {<Parameter "swapStyle: Literal['innerHTML', 'outerHTML', 'testContent', 'beforebegin', 'afterbegin', 'beforeend', 'afterend', 'delete', 'none']">: 'innerHTML',
  <Parameter "swapDelay">: 1000}}

----
# HTMXCommander plugin

We can use `HTMXCommander` by itself. But probably, it'll be more useful as a bridge plugin.

In [ ]:
#| export

class HTMXCommanderPlugin(BridgePlugin):
    src = bundled('''
import { setupCommander, onCommanderMsg } from './commander.js';

export default async function initializeCommander(bridge) {
    const sels = await bridge.get('commander', 'output_sels');
    await setupCommander(sels);
    bridge.on("commander", onCommanderMsg);
    return () => bridge.off('commander');
}
''')()
    ctx_name = 'commander'
    output_sels = ['.output', '.jp-Cell-outputArea']

    _bridge_delegates = ['swap']
    def on_inspect(self, *args, msg:Any, tracker:Any, **kwargs):
        self.log(f"{self.__class__.__name__} inspect: {msg=} {tracker=}")
    
    # hate stupid wiggly reds
    def swap(self: HTMXCommanderPlugin,
        target, 
        content, 
        *, 
        # ---- swapSpec:SwapSpec, 
        swapStyle: Literal['innerHTML','outerHTML','testContent','beforebegin','afterbegin','beforeend','afterend','delete','none'],
        swapDelay: int|None=None, settleDelay: int|None=None,
        transition: bool|None=None,
        # ignoreTitle: bool|None=None, head: Literal['merge', 'append']|None=None,
        scroll: str|None=None, scrollTarget: str|None=None,
        show: str|None=None, showTarget: str|None=None, focusScroll: bool|None=None,
        # ---- swapOptions=None,
        select: str|None=None, selectOOB: str|None=None,
        # eventInfo: dict|None=None,
        anchor: str|None=None,        
        # contextElement: str|None=None,
        # afterSwapCallback: Callable|None=None, afterSettleCallback: Callable|None=None,
    ): ...

FC.patch_to(HTMXCommanderPlugin)(swap)

In [ ]:
bridge.add_plugins((cmdr := HTMXCommanderPlugin()), wait=5)

In [ ]:
(d := Div(id='output-9999')('Original 2'))

div(('Original 2',),{'id': 'output-9999'})

In [ ]:
cmdr.swap(d, Div('Swapped 2'), swapStyle='innerHTML')

In [ ]:
def swap(self,*args, **kwargs): 
    bridge.commander.swap(self, *args, **kwargs)
FC.patch_to(FT)(swap)

In [ ]:
div = Div(id='output-99999')('Original 3')
div

div(('Original 3',),{'id': 'output-99999'})

In [ ]:
div.swap('<div>Swapped 3</div>', swapStyle='innerHTML')

# NBHooksPlugin
> Python-only bridge plugin to help us inspect, control, and modify cell outputs.

Note: in VSCode output_capture is unreliable, unfortunately, and don't play well with the debugger. Jeremy rules in almost everything programming-wise, but in this I'm more aligned with Carson: [Grug Brained Developers](https://grugbrain.dev/) need powerful (graphical, I double Grug) debuggers. How very much I would rather not use VSCode notebooks.

You can switch on/off the capturer. We'll hopefully be able to develop alternatives down the road.

In [ ]:
#| export

class NBHooksPlugin(BridgePlugin):
    ctx_name = 'nbhooks'
    _bridge_delegates = ['dh', 'dhs']
    def __init__(self, *args, **kwargs):
        # Warning: ensure this is called from the main thread
        self.csi, self.brdd, self.capturer = get_csi(True), get_bridged(True), get_capturer(True)
        super().__init__(*args, **kwargs)
    
    def on_init(self, *args, info:str, **kwargs):
        super().on_init(*args, info=info, **kwargs)
        if self.is_initialized is True: self.bridge.state = get_nb_from_hooks()

    @property
    def dh(self)->DisplayHandle: return self.brdd.dh  # type: ignore

In [ ]:
bridge_cfg.update(auto_id=True, auto_show=True)

{'auto_show': True, 'auto_mount': False, 'auto_id': True, 'bundle_cfg': {'out_dir': [Path('/Users/vic/dev/repo/project/bridget/bridget/js'), Path('/Users/vic/dev/repo/project/bridget/bridget')], 'rewrite_imports': True, 'import_name': 'brdimport'}, 'bootstrap': False, 'current_did': None}

In [ ]:
bridge.add_plugins(NBHooksPlugin())

In [ ]:
cprint(bridge.nbhooks.brdd.dhs)

deque(maxlen=100)

In [ ]:
HTML(f"2, 3, 5, 7")


In [ ]:
test_eq(__nb__[__lastcellinfo__.cell_id].outputs[0].metadata['brd_did'], bridge.nbhooks.dh.display_id)
__nb__[__lastcellinfo__.cell_id]

{'idx': 1,
 'source': 'HTML(f"2, 3, 5, 7")\n',
 'id': 'Y103sZmlsZQ==',
 'cell_type': 'code',
 'outputs': [{'output_type': 'display_data',
   'data': {'text/plain': '<bridget.helpers.HTML object>',
    'text/html': '2, 3, 5, 7\n<brd-mark id="b18fb6ab5-07024c14-938dbe40-64c426d9"></brd-mark>'},
   'metadata': {'bridge': {'captured': True},
    'brd_did': 'b18fb6ab5-07024c14-938dbe40-64c426d9'}},
  {'output_type': 'execute_result',
   'execution_count': 41,
   'data': {'text/plain': '<bridget.helpers.HTML object>',
    'text/html': '2, 3, 5, 7'},
   'metadata': {}}],
 'execution_count': 41}

In [ ]:
bridge.nbhooks.brdd.dhs[-2].update(HTML(f"11, 13, 17, 19, 23, 29"))
# cprint(__cellinfo__)

In [ ]:
ci = __cellinfo__
displaydh(HTML(f"It’s a fact everybody already knows,<br>Single rich man, man he gotta propose."))
HTML(' ')

In [ ]:
bridge.nbhooks.dh.update(HTML(f"New in town, what’s he thinking? Nobody cares,<br>Every mama scheming, whisperin’ prayers."))
cprint(__cellinfo__)

{
    'source': 'bridge.nbhooks.dh.update(HTML(f"New in town, what’s he thinking? Nobody cares,<br>Every mama scheming, whisperin’ 
prayers."))\ncprint(__cellinfo__)',
    'cell_id': 'Y110sZmlsZQ==',
    'exec_result': {'result': None}
}

In [ ]:
speech_Batty = """I've seen things you people wouldn't believe.
Attack ships on fire off the shoulder of Orion.
I watched C-beams glitter in the dark near the Tannhäuser Gate.
All those moments will be lost in time, like tears in rain.
Time to die.""".split('\n')

HTML('<br>'.join(speech_Batty[:-2]) + "<div class='what-follows' style='color: red;'>...</div>")

In [ ]:
bridge.commander.swap(
    f"div[data-brt-id={bridge.nbhooks.dh.display_id}]>.what-follows", 
    Div(style={'color':'darkgreen'})(speech_Batty[-2], Br(), speech_Batty[-1]),
    swapStyle='outerHTML')

In [ ]:
bridge.nbhooks.dh.update(HTML('<br>'.join(speech_Batty)))

# &lt;zero-md&gt;
> Ridiculously simple zero-config markdown displayer

In [ ]:
md_samp = '''
# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```
'''

Markdown(md_samp)


# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```


In [ ]:
#| export

zeromd_js = '''
import ZeroMd from 'https://cdn.jsdelivr.net/npm/zero-md@3'
customElements.define('zero-md', ZeroMd);
'''

zeromd_scr = Script(debug(zeromd_js), id='zeromd', type='module')

In [ ]:
bridge.logger.show()

In [ ]:
bridge.loader.load_links({'zeromd':zeromd_scr})

In [ ]:
from fasthtml.components import Zero_md, Template
from bridget.bridge_widget import StyleV

def render_local_md(md, css=''):
    css_template = Template(StyleV(css), data_append=True)
    return Zero_md(css_template, Script(md, type="text/markdown"))

In [ ]:
md_ft = render_local_md(md_samp)
with bridge_cfg(auto_show=False): display(md_ft)

```html
<zero-md><template data-append>    <style></style>
</template><script type="text/markdown">
# &lt;zero-md&gt;

> Ridiculously simple zero-config markdown displayer

A vanilla markdown-to-html web component based on
[Custom Elements V1 specs](https://www.w3.org/TR/custom-elements/) to load and display an external
MD file.

Featuring:

- [x] Math rendering via [`KaTeX`](https://github.com/KaTeX/KaTeX)
- [x] [`Mermaid`](https://github.com/mermaid-js/mermaid) diagrams
- [x] Syntax highlighting via [`highlight.js`](https://github.com/highlightjs/highlight.js)

```python
a = {'a': 'asdf', 'b': 10}
print("local server is listening on port 8080")
```
</script></zero-md>
```

In [ ]:
md_ft

zero-md((template((style(('',),{}),),{'data-append': True}), script(('## Hi again!',),{'type': 'text/markdown'})),{})

In [ ]:
bridge.nbhooks.dh.update(render_local_md('## Hi again!'))

# Recap of NBHooks features:

As a bridge plugin, NBHooks provides:

1. Wrapper of CellExecInfo to provide current and last cell info, including cell id.
2. Add metadata to all displayed rich media
3. Add metadata to cell output.

In [ ]:
cleanupwidgets('bridge')

bridge = get_bridge(show_logger=True)
bridge.add_plugins(NBHooksPlugin())

### 1. Capture cell info
>  with cell id if possible: Jupyter Notebook/Lab, VSCode-ish; nbclassic doesn't gen cell ids.

Will be handy when we have access to notebook state.

In [ ]:
cprint(ci := __cellinfo__)
test_eq(__cellinfo__['source'][:26], 'cprint(ci := __cellinfo__)')
# test_eq(brd.csi.dh['cell_id'], cid)

{
    'source': "cprint(ci := __cellinfo__)\ntest_eq(__cellinfo__['source'][:26], 'cprint(ci := __cellinfo__)')\n# 
test_eq(brd.csi.dh['cell_id'], cid)",
    'cell_id': 'Y132sZmlsZQ==',
    'exec_result': {'result': None}
}

### 2. Every rich cell output, those made with display(...), or last cell expression, is captured and converted to transient

i.e., equivalent to display(...,display_id=True).  
NBHooks stores the `DisplayHandle`, and __cellinfo__ got the `display_id` in `__cellinfo__.did`

In [ ]:
display(Markdown("""`The World in His Arms`"""))

In [ ]:
bridge.nbhooks.dh.update(A(href='https://en.wikipedia.org/wiki/The_World_in_His_Arms')('The World in His Arms'))

In [ ]:
bridge.nbhooks.dh.update(Image(url='https://upload.wikimedia.org/wikipedia/commons/4/4b/WorldInHisArms-poster.jpg', height=150))

Capture automatic cell output also

In [ ]:
Div('asdf')

In [ ]:
# bridge.nbhooks.dh.update(HTML('<div>qwer</div>'))
bridge.nbhooks.dh.update(HTML(Div('qwer')))

### 3. All HTML captured outputs have Bridge metadata (see Brd_Mark)

We use a custom element to mark the parent element in the front-end. HTML display objects allow us to add this custom element (brd_mark) easily. We'd also like to tag other rich outputs but we can't do it with only IPython/Kernel mechanisms.

In [ ]:
did = '23579111317'
HTML(Div(f"My parent element has a `data-brd-id` attribute with value '{did}'. Inspect me!"), 
    metadata={'brd_did': did})

In [ ]:
test_eq(__nb__[__lastcellinfo__.cell_id].outputs[0]['data']['text/html'].split('\n')[-1], f'<brd-mark id="{did}"></brd-mark>')

### 4. HTMX API (WIP)

Besides DisplayHandler.update(...) you can modify the HTML outputs directly with HTMX API python wrapper (swap only for now).

# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean
from olio.basic import bundle_path
from bridget.bridge_widget import bundled

In [ ]:
if FC.IN_NOTEBOOK:
    BUNDLE_PATH = bundle_path(__name__)
    for f in ['commander']: bundled(BUNDLE_PATH / f'js/{f}.js')()
    nb_path = '16_bridge_plugins.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)